<span>
<!-- <img src="http://ndlib.readthedocs.io/en/latest/_static/ndlogo2.png" width="260px" align="right"/> -->
</span>
<span>
<b>Author:</b> <a href="http://about.giuliorossetti.net">Giulio Rossetti</a><br/>
<b>Python version:</b>  >=3.6<br/>
<b>DyNetX version:</b>  0.0.1<br/>
<b>Last update:</b> 15/02/2021
</span>

<a id='top'></a>
# *Chapter 9: Network Dynamics*

In this notebook are introduced basilar snippet to cover dynamic networ modeling and analysis.

**Note:** this notebook is purposely not 100% comprehensive, it only discusses the basic things you need to get started. 

In [1]:
import dynetx as dn
import networkx as nx
import random

g = dn.DynGraph() # empty dynamic graph

## Dynamic network creation

A dynamic network can be built by adding edges with specific appearence time (and eventually, vanishing time).

In our example, 10 ER graphs are generated and used to represent different topological evolutions of a same dynamic system.

In [2]:
for t in range(0, 10):
    er = nx.erdos_renyi_graph(random.randint(100, 400), 0.05)
    g.add_interactions_from(er.edges, t=t)

We can get the list of snapshot ids with

In [3]:
g.temporal_snapshots_ids()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

Moreover, we can access each snapshot by its id

In [4]:
g1 = g.time_slice(1)

In [5]:
type(g1), g1.number_of_nodes(), g1.number_of_edges()

(dynetx.classes.dyngraph.DynGraph, 148, 570)

Following the same rationale it is possible to obtain timeslices covering any time window

In [6]:
g0_3 = g.time_slice(0, 3)

In [7]:
type(g0_3), g0_3.number_of_nodes(), g0_3.number_of_edges(), g0_3.interactions_per_snapshots()

(dynetx.classes.dyngraph.DynGraph, 328, 6954, {0: 10.5, 1: 16.0, 2: 39.5})

If the slice cover a single snapshot it can be analyzed transforming it in a ``networkx`` object, otherwise ``dynetx`` methods need to be applied

In [8]:
g1_flat = nx.Graph(g1.edges())

In [9]:
type(g1_flat), g1_flat.number_of_nodes(), g1_flat.number_of_edges()

(networkx.classes.graph.Graph, 148, 570)

### Dynamic network measures

#### Inter event time (Global)

Distribution of inter event time (e.g., how much time before a new interaction appears in the graph)

In [10]:
r = g.inter_event_time_distribution()
print(f"Number interactions: temporal distance\t{r}")

Number interactions: temporal distance	{0: 15176, 1: 10}


#### Inter event time (Node)

Distribution of inter event time (e.g., how much time before a new interaction involving a specific node appears in the graph)

In [11]:
r = g.inter_event_time_distribution(0)
print(f"Number interactions: temporal distance\t{r}")

Number interactions: temporal distance	{0: 96, 1: 9}


#### Inter event time (Edge)

Distribution of inter event time (e.g., how much time before a new interaction among two nodes, u and v, appears in the graph)

In [12]:
u, v, _ = g.interactions()[0] # selecting two nodes for which at least an interaction exists

In [13]:
r = g.inter_event_time_distribution(u, v)
print(f"Number interactions: temporal distance\t{r}")

Number interactions: temporal distance	{}


### Degree
Degrees can be queried time-wise

In [14]:
g.degree(t=2)[0] # degree of node 0 at time t=2

10

### Coverage

The ratio of existing nodes w.r.t. the possible ones.

In [15]:
g.coverage()

0.7304878048780488

#### Node contribution

Node u coverage of the temporal graph.

In [16]:
g.node_contribution(250)

0.6

#### Edge contribution

Edge (u, v) coverage of the temporal graph.

In [17]:
g.edge_contribution(u, v)

0.1

#### Node pair uniformity

Overlap between the presence times of u and v.

In [18]:
g.node_pair_uniformity(0, 250)

0.6

### Density
Temporal network density: fraction of possible interactions that do exist in the temporal network.

In [19]:
g.density()

0.05073807027169167

#### Node Density
Intersection among the temporal presence of the edge (u, v) and the joint temporal presences of u and v.

In [20]:
g.node_density(u)

0.04674457429048414

#### Pair Density

Intersection among the temporal presence of the edge (u, v) and the joint temporal presences of u and v.

In [21]:
g.pair_density(u, v)

0.1

#### Snapshot Density

Density of a temporal network at time t.

In [22]:
for t in g.temporal_snapshots_ids():
    print(f"{t}\t{g.snapshot_density(t)}")

0	0.05016036398896099
1	0.05239933811362383
2	0.05085425523381728
3	0.05153890824622532
4	0.05118497766336034
5	0.051248299767452064
6	0.050530376084860176
7	0.05209627329192547
8	0.04892425582080755
9	0.05072382448658961


### Path analysis

Computes the time respecting paths among u and v within [start, stop]

In [23]:
import dynetx.algorithms as al
paths = al.time_respecting_paths(g, 0, 10, start=0, end=3)

In [24]:
p = paths[0] # example of identified paths. Each list element is a tuple of the form (from, to, time)
p

[(0, 140, 0), (140, 10, 1)]

Moreover, it is possible to compute length and duration of a given path

In [25]:
al.path_duration(p), al.path_length(p)

(1, 2)

Among all paths it is possible to identify the most interestin ones using

In [26]:
annotated = al.annotate_paths(paths)

In [27]:
annotated

{'shortest': [[(0, 10, 2)], [(0, 10, 2)]],
 'fastest': [[(0, 10, 2)], [(0, 10, 2)]],
 'shortest_fastest': [[(0, 10, 2)]],
 'fastest_shortest': [[(0, 10, 2)]],
 'foremost': [[(0, 140, 0), (140, 10, 1)]]}